# Neural Net Game of Life

A grid of cells, each one has an hp value as a float between 0 and 1, and a color RGB.
each turn, the cell picks a color for itself, and tries to predict the color of all of its neighbors
cell gains hp based on how accurately it predicted neighbors colors
cell loses hp if neighbors predicted its color
a successful color prediction is a "chomp"

it needs memory! use a recursive network. need to use multiclass cross entropy loss function for learning.

cell takes more damage if a high hp cell chomps it?

cells with super high fitness copy their weights over to another cell?

hostile environment? all cells take damage over time unless they act correctly to survive?

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
import numpy as np

In [ ]:
class Cell():
    def __init__(self):
        self.hp = 0.0
        self.color = 'red'
        self.brain = nn.Sequential()

    def set_hp(self, hp):
        self.hp = hp
    def set_color(self, color):
        self.color = color
    def __repr__(self):
        pass
        # print the state of the cell
